In [ ]:
#Sample Input File (titanic.csv):
PassengerId,Survived,Pclass,Name,Sex,Age
1,0,3,John,male,22
2,1,1,Mary,female,38
3,1,3,Anna,female,26
4,0,1,Bob,male,35
5,0,2,Lily,female,30
6,0,3,Susan,female,18

In [ ]:
#!/usr/bin/env python3
import sys

for line in sys.stdin:
    if line.startswith("PassengerId"):
        continue
    fields = line.strip().split(',')
    if len(fields) < 6:
        continue

    survived = fields[1]
    pclass = fields[2]
    sex = fields[4].lower()
    age = fields[5]

    if survived == '0':  # only for those who died
        if sex == 'male' and age:
            print("male\t" + age)
        elif sex == 'female':
            print(f"class_{pclass}\t1")


In [ ]:
#!/usr/bin/env python3
import sys

male_age_sum = 0
male_count = 0
class_deaths = {}

for line in sys.stdin:
    key, value = line.strip().split('\t')

    if key == "male":
        try:
            age = float(value)
            male_age_sum += age
            male_count += 1
        except ValueError:
            continue
    elif key.startswith("class_"):
        if key in class_deaths:
            class_deaths[key] += 1
        else:
            class_deaths[key] = 1

if male_count > 0:
    avg_age = male_age_sum / male_count
    print(f"Average age of deceased males: {avg_age:.2f}")

for cls in sorted(class_deaths):
    print(f"{cls.replace('class_', 'Class ')} female deaths: {class_deaths[cls]}")


In [ ]:
#Execution Commands:


# Upload file to HDFS
hadoop fs -put titanic.csv /input/titanic.csv
# Run MapReduce job
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-files mapper.py,reducer.py \
-mapper mapper.py \
-reducer reducer.py \
-input /input/titanic.csv \
-output /output_titanic
# View output
hadoop fs -cat /output_titanic/part-00000